# Kaggle Bike Sharing Demand(レンタサイクルの需要予測)

## データの読み込み

In [372]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # グラフ描画用
import seaborn as sns; sns.set() # グラフ描画用
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error # モデル評価用(平均二乗誤差)
from sklearn.metrics import r2_score # モデル評価用(決定係数)

In [373]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
df = pd.read_csv('output/df_prepro.csv')
submission = pd.read_csv('input/sampleSubmission.csv')
display(df.head())
# display(df.info())

,year,month,weekday,hour,day_type,weather,temp,humidity,windspeed,casual,registered,casual_mov_ave,registered_mov_ave
0,2011,1,2,0,1,1,9.84,81,0.0,3.0,13.0,NaN,NaN
1,2011,1,2,1,1,1,9.02,80,0.0,8.0,32.0,NaN,NaN
2,2011,1,2,2,1,1,9.02,80,0.0,5.0,27.0,NaN,NaN
3,2011,1,2,3,1,1,9.84,75,0.0,3.0,10.0,NaN,NaN
4,2011,1,2,4,1,1,9.84,75,0.0,0.0,1.0,NaN,NaN


In [374]:
df['count'] = df['casual'] + df['registered']
df = df[['temp','humidity','hour','count']]

In [375]:
df.columns

Index(['temp', 'humidity', 'hour', 'count'], dtype='object')

## 対数変換

In [376]:
def y_transform(df, col):
    # df[col] = np.sqrt(df[col])
    count_min  = df.loc[df[col].values > 0, col].min() #0を除いた最小値
    df[col] = train[col] + count_min
    df[col] = np.log(df[col])
    
# log_transform('temp')
# log_transform('humidity')
# log_transform('windspeed')
# y_transform(df, 'casual_mov_ave')
# y_transform(df, 'registered_mov_ave')
# y_transform(df, 'casual')

# count_min = df.loc[df['casual'].values > 0, 'casual'].min() #0を除いた最小値
# df['casual'] = df['casual'] + count_min
# df['casual'] = np.log(df['casual'])


# df['casual'].describe()

## 機械学習
trainデータを分割してcasualとregisteredを目的変数に設定

In [369]:
# df.drop(['registered_mov_ave'], axis=1, inplace=True)
train = df.iloc[:len(train),:].copy()
test  = df.iloc[len(train):,:].copy()
test.drop(['count'], axis=1, inplace=True)

### casualを学習

In [370]:
X = train.drop(['count'], axis=1)
y = train['count']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33, random_state=0)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

from sklearn.linear_model import Lasso                         # LASSO回帰のクラスを読み込み
model_lasso = Lasso(alpha=10)                                # LASSO回帰のインスタンスを生成
model_lasso.fit(X_train, y_train)                                      # データを学習させる



Lasso(alpha=10)

In [371]:
def rmsle(real, predicted):
    sum=0.0
    for x in range(len(predicted)):
        if predicted[x]<0 or real[x]<0: #check for negative values
            continue
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

print("test: ",rmsle(y_test, lr.predict(X_test)))
print("train: ",rmsle(y_train, lr.predict(X_train)))


KeyError: 0

In [336]:
# test_pred['casual'] = test_pred['casual']*count_max
# test_pred['casual'] = test_pred['casual']

# count_min = df.loc[df['casual'].values > 0, 'casual'].min() #0を除いた最小値(=1)
# count_max = df['casual'].max()
# df['casual'] = df['casual'] + count_min
# df['casual'] = df['casual']/count_max #0~1になる

-9.98483165182296

In [337]:
# def inv_transform(trg, count_min):
#     count = machine_learning(trg)[trg]
#     count = np.exp(count) - count_min
#     return count

In [338]:
# submission['count'] = inv_transform('casual', count_min) + inv_transform('registered', count_min)
# submission['count'] = round(submission['count'],0).astype('int')

In [339]:
# submission.describe()

In [340]:
# submission.to_csv('output/submission.csv',index=False)

In [341]:
# df.to_csv('output/df_prepro.csv',index=False)